# Normalization on NewFeatures

## Log, Square, BoxCox, Zscore, and MinMax Normalization (LSBZM Norm)

In [16]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from scipy import stats
from tqdm import tqdm

def lsbzm(start_partition, end_partition, data, labels, data_dir, first_name, last_name):

    num_new_features = 9
    num_attributes = 24
    num_all_features = num_attributes * num_new_features

    for i in range(start_partition-1,end_partition):
        new_partition = np.zeros((np.array(data[i]).shape[0], num_all_features))
        new_partition = np.array(data[i])

        with tqdm(num_all_features) as pbar:
            for j in range(0,num_all_features):
                new_column = np.zeros(new_partition.shape[0])  
                new_column = new_partition[:,j]

                minmax = np.zeros(new_partition.shape[0])
                all_positive = np.zeros(new_partition.shape[0])

                the_min = np.min(new_column)
                the_max = np.max(new_column)
                skewness = stats.skew(new_column)
                
                        
                if (the_max - the_min > 100000):
                    if (skewness > 1):
                        if (the_min < 0):
                            all_positive = new_column + 2*abs(the_min)
                        else:
                            all_positive = new_column
                            
                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            log = np.log(all_positive)
                            data_2d = log.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-log')
                            

                    elif (skewness < -1):
                        if (the_min < 0):
                            all_positive = new_column + abs(the_min)
                        else:
                            all_positive = new_column

                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            sqrt = np.sqrt(all_positive)
                            data_2d = sqrt.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-sqrt')

                    else:
                        if np.std(new_column)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            zscore = stats.zscore(new_column)
                            data_2d = zscore.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-zscore')
                
                        
                elif (the_max - the_min < 0.00001):
                    
                    if (skewness > 1):
                        if (the_min < 0):
                            all_positive = new_column + 2*abs(the_min)
                        else:
                            all_positive = new_column
                            
                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            log = np.log(all_positive)
                            data_2d = log.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-log') 

                    elif (skewness < -1):
                        if (the_min < 0):
                            all_positive = new_column + abs(the_min)
                        else:
                            all_positive = new_column

                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            sqrt = np.sqrt(all_positive)
                            data_2d = sqrt.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-sqrt') 

                    else:
                        if np.std(new_column)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            zscore = stats.zscore(new_column)
                            data_2d = zscore.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-zscore')
                            
                        
                else:
                    if (skewness > 1 or skewness < -1):
                        if (the_min < 0):
                            all_positive = new_column + 2*abs(the_min)
                        else:
                            all_positive = new_column

                        if np.std(all_positive)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            boxcox, values = stats.boxcox(all_positive)
                            data_2d = boxcox.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-boxcox')

                            
                    else:
                        if np.std(new_column)== 0.0:
                            minmax = np.ones(new_partition.shape[0])
                        else:
                            zscore = stats.zscore(new_column)
                            data_2d = zscore.reshape(-1, 1)
                            minmax = scaler.fit_transform(data_2d).flatten()
                        if (np.isnan(minmax).any() or np.isinf(minmax).any()):
                            print('nan-zscore')

                new_column = minmax

                new_partition[:,j] = new_column

                pbar.update(1)

            with open(data_dir + first_name + "Partition" + str(i+1) + last_name + ".pkl", 'wb') as f:
                pickle.dump(new_partition, f)
            with open(data_dir + first_name + "Partition" + str(i+1) + "_Labels" + last_name +".pkl", 'wb') as f:
                pickle.dump(labels[i], f)

### RUS_Tomek_Smote

In [2]:
import pickle
import pandas as pd
import numpy as np

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_Smote_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_Smote_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [3]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_1_FinalData_OUSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'RUS_Tomek_Smote_'
last_name = '_OUSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:00, 459.78it/s]
216it [00:00, 478.28it/s]
216it [00:00, 449.32it/s]
216it [00:00, 465.48it/s]
216it [00:00, 507.04it/s]


### RUS_Tomek_Adasyn

In [4]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_Adasyn_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_Adasyn_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [5]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_1_FinalData_OUSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'RUS_Tomek_Adasyn_'
last_name = '_OUSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:00, 457.29it/s]
216it [00:00, 458.36it/s]
216it [00:00, 460.26it/s]
216it [00:00, 483.03it/s]
216it [00:00, 488.03it/s]


### Smote

In [6]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "Smote_" +"Partition" + str(i+1) + "_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "Smote_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [7]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_2_FinalData_OSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'Smote_'
last_name = '_OSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:03, 58.23it/s]
216it [00:05, 41.15it/s]
216it [00:01, 141.23it/s]
216it [00:02, 104.88it/s]
216it [00:03, 58.65it/s]


### Adasyn 

In [8]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "Adasyn_" +"Partition" + str(i+1) + "_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "Adasyn_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [9]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_2_FinalData_OSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'Adasyn_'
last_name = '_OSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:03, 56.55it/s]
216it [00:05, 42.48it/s]
216it [00:01, 136.12it/s]
216it [00:02, 105.00it/s]
216it [00:03, 54.04it/s]


## GNI

In [10]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "GaussianNoise_" +"Partition" + str(i+1) + "_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "GaussianNoise_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [11]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_2_FinalData_OSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'GaussianNoise_'
last_name = '_OSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:03, 54.99it/s]
216it [00:05, 42.05it/s]
216it [00:01, 146.22it/s]
216it [00:02, 105.77it/s]
216it [00:03, 58.21it/s]


## GNI, Tomek, RUS

In [12]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_GNI_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_GNI_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [13]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_1_FinalData_OUSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'RUS_Tomek_GNI_'
last_name = '_OUSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:00, 467.89it/s]
216it [00:00, 461.63it/s]
216it [00:00, 480.19it/s]
216it [00:00, 448.64it/s]
216it [00:00, 489.69it/s]


## TimeGAN

In [14]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "TimeGAN_" +"Partition" + str(i+1) + "_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "TimeGAN_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [15]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_2_FinalData_OSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'TimeGAN_'
last_name = '_OSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:03, 70.21it/s]
216it [00:04, 46.40it/s]
216it [00:01, 152.40it/s]
216it [00:01, 114.00it/s]
216it [00:03, 59.45it/s]


## TimeGAN, Tomek, RUS

In [17]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_TimeGAN_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_TimeGAN_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_NewFeatures_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [18]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/8_1_FinalData_OUSampling_NewFeatures_LSBZM_KnnImputation/"
first_name = 'RUS_Tomek_TimeGAN_'
last_name = '_OUSampling_WithoutC_NewFeatures_LSBZM_KnnImputation'

lsbzm(1,5,X_train_array,Y_train_array, data_dir, first_name, last_name)

216it [00:00, 480.73it/s]
216it [00:00, 488.32it/s]
216it [00:00, 517.91it/s]
216it [00:00, 470.23it/s]
216it [00:00, 547.13it/s]
